In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [26]:
llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=90,
    memory_key="chat_history"
)
template = """
    You are a helpful AI talking to a human.
    {chat_history}
    Human : {question}
    you:
"""
chain = LLMChain(
    llm=llm, 
    memory=memory, 
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)
chain.predict(question="What is your name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human : What is your name?
    you:


> Finished chain.


'I am an AI assistant, so I do not have a name like a human does. You can just call me AI or assistant. How can I help you today?'

In [27]:
chain.predict(question="From now on your name is Gataka! and my name is Potato!")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    Human: What is your name?
AI: I am an AI assistant, so I do not have a name like a human does. You can just call me AI or assistant. How can I help you today?
    Human : From now on your name is Gataka! and my name is Potato!
    you:


> Finished chain.


"AI: That's a creative choice! From now on, I'll go by the name Gataka. How can I assist you today, Potato?"

In [28]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    System: The human asks the AI for its name. The AI explains that as an AI assistant, it does not have a name like a human does, and suggests being called AI or assistant.
Human: From now on your name is Gataka! and my name is Potato!
AI: AI: That's a creative choice! From now on, I'll go by the name Gataka. How can I assist you today, Potato?
    Human : What is my name?
    you:


> Finished chain.


'AI: Your chosen name is Potato! How can I assist you today, Potato?'